# PSO - Finding hyper Paramters

In [47]:
import itertools
import numpy as np
from optimisers import PSO
from trading_bot import TradingBotOptimiser  

class PSOGridSearch:
    def __init__(self, dataset_path, param_grid, iterations):
        self.dataset_path = dataset_path
        self.param_grid = param_grid
        self.iterations = iterations
        self.best_score = -np.inf
        self.best_params = None

    def run(self):
        param_combinations = list(itertools.product(*self.param_grid.values()))

        for params in param_combinations:
            rng_seed, population_size, p_increment, g_increment = params

            # Create the PSO optimiser with current params
            optimiser = PSO(
                rng_seed=int(rng_seed), 
                population_size=int(population_size),
                p_increment=p_increment, 
                g_increment=g_increment
            )
            
            bot_optim = TradingBotOptimiser(self.dataset_path, optimiser)
            
            try:
                _, score = bot_optim.optimise(self.iterations)
            except Exception as e:
                print(f"Failed on params {params}: {e}")
                continue

            if score > self.best_score:
                self.best_score = score
                self.best_params = params

            print(f"Tested params {params} -> Score: {score:.2f}")

        return self.best_params, self.best_score

# Define grid to search
param_grid = {
    'rng_seed': [42],
    'population_size': [5, 10, 20],
    'p_increment': [0.05, 0.1, 0.2],
    'g_increment': [0.05, 0.1, 0.2]
}

if __name__ == "__main__":
    dataset_path = './trainingDaily.csv' 
    grid_search = PSOGridSearch(dataset_path, param_grid, iterations=100) 
    best_params, best_score = grid_search.run()

    print("\nBest PSO Parameters:", best_params)
    print("Best TradingBot Score:", best_score)


iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Tested params (42, 5, 0.05, 0.05) -> Score: 26222.20
iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Tested params (42, 5, 0.05, 0.1) -> Score: 21734.08
iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Tested params (42, 5, 0.05, 0.2) -> Score: 19400.15
iteration

In [22]:
#Best PSO Parameters: (42, 10, 0.2, 0.1)
#Best TradingBot Score: 31353.24571724603

# PSO -Training

In [26]:
from optimisers import PSO

class TrackingPSO(PSO):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.score_history = []

    def step(self):
        super().step()
        _, best_fitness = self.current_best()
        self.score_history.append(best_fitness)


In [45]:
from optimisers import PSO
from trading_bot import TradingBotOptimiser

best_pso_params = {
    'rng_seed': 42,
    'population_size': 20, #10,
    'p_increment': 2, #0.2,
    'g_increment': 0.1
}

train_dataset_path = './trainingDaily.csv'  

pso_optimiser = TrackingPSO(
    rng_seed=best_pso_params['rng_seed'],
    population_size=best_pso_params['population_size'],
    p_increment=best_pso_params['p_increment'],
    g_increment=best_pso_params['g_increment']
)

pso_optim = TradingBotOptimiser(train_dataset_path, pso_optimiser)
pso_best_pos, pso_best_fitness = pso_optim.optimise(100)  
pso_score_std = np.std(pso_optimiser.score_history)

print("\nTrain Set Evaluation:")
print("Best PSO Parameters:", best_pso_params)
print("Train Set Score:", pso_best_fitness)
print(f"  PSO: score {pso_best_fitness:.8g} at pos ({pso_best_pos})")
print("Standard Deviation of Scores:", pso_score_std)

iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95

Train Set Evaluation:
Best PSO Parameters: {'rng_seed': 42, 'population_size': 20, 'p_increment': 2, 'g_increment': 0.1}
Train Set Score: 41699.41506597269
  PSO: score 41699.415 at pos ([ 1.00000000e+02  1.00000000e+02  9.00000000e+01  1.00000000e-10
 -7.66617168e+01  4.99693910e+01 -2.47984170e+00  1.78569465e+01
 -2.25947440e+01  0.00000000e+00])
Standard Deviation of Scores: 6714.721711276471


Train Set Evaluation:
Best PSO Parameters: {'rng_seed': 42, 'population_size': 10, 'p_increment': 0.2, 'g_increment': 0.1}
Train Set Score: 31353.24571724603
  PSO: score 31353.246 at pos ([ 6.50000000e+01  7.10000000e+01  5.40000000e+01  1.00000000e-10
 -4.55486260e+01 -2.69761667e+01 -5.44823753e+01 -2.60940676e+01
 -5.12077242e+01  6.64248561e+01])
Standard Deviation of Scores: 6537.159562545172

## PSO -Test

In [46]:

from trading_bot import TradingBotInstance

test_dataset_path = './testDaily.csv'

# Use the best parameters found from training
pso_bot = TradingBotInstance(test_dataset_path, *pso_best_pos)

# Run simulation on test set
pso_score = pso_bot.simulate_run()

print("\nTest Set Evaluation Using Best PSO Parameters:")
print("Parameters:", pso_best_pos)
print("Score:", pso_score)



Test Set Evaluation Using Best PSO Parameters:
Parameters: [ 1.00000000e+02  1.00000000e+02  9.00000000e+01  1.00000000e-10
 -7.66617168e+01  4.99693910e+01 -2.47984170e+00  1.78569465e+01
 -2.25947440e+01  0.00000000e+00]
Score: 2728.0299497914525


Test Set Evaluation Using Best PSO Parameters:
Parameters: [ 6.50000000e+01  7.10000000e+01  5.40000000e+01  1.00000000e-10
 -4.55486260e+01 -2.69761667e+01 -5.44823753e+01 -2.60940676e+01
 -5.12077242e+01  6.64248561e+01]
Score: 2374.4240665538146

# PSO-SA  - Finding hyper Paramters

In [31]:
import itertools
import numpy as np
from trading_bot import TradingBotOptimiser
from optimisers import PSOSA

class PSOSAGridSearch:
    def __init__(self, dataset_path, param_grid, iterations):
        self.dataset_path = dataset_path
        self.param_grid = param_grid
        self.iterations = iterations
        self.best_score = -np.inf
        self.best_params = None

    def run(self):
        keys = list(self.param_grid.keys())
        param_combinations = list(itertools.product(*self.param_grid.values()))

        for values in param_combinations:
            params = dict(zip(keys, values))

            optimiser = PSOSA(
                rng_seed=params['rng_seed'],
                population_size=params['population_size'],
                p_increment=params['p_increment'],
                g_increment=params['g_increment'],
                iters=params['iters']  
            )

            bot_optim = TradingBotOptimiser(self.dataset_path, optimiser)

            try:
                # Pass 'iter' explicitly as required
                _, score = bot_optim.optimise(self.iterations)
            except Exception as e:
                print(f"Failed with {params}: {e}")
                continue

            if score > self.best_score:
                self.best_score = score
                self.best_params = params

            print(f"Params: {params} -> Score: {score:.2f}")

        return self.best_params, self.best_score


# Example usage
if __name__ == "__main__":
    param_grid = {
        'rng_seed': [42],
        'population_size': [5, 10],
        'p_increment': [0.05, 0.1],
        'g_increment': [0.05, 0.1],
        'iters': [100]
    }

    dataset_path = './trainingDaily.csv'
    search = PSOSAGridSearch(dataset_path, param_grid,100)
    best_params, best_score = search.run()

    print("\nBest PSOSA Hyperparameters:", best_params)
    print("Best Score:", best_score)


iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Params: {'rng_seed': 42, 'population_size': 5, 'p_increment': 0.05, 'g_increment': 0.05, 'iters': 100} -> Score: 15528.03
iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Params: {'rng_seed': 42, 'population_size': 5, 'p_increment': 0.05, 'g_increment': 0.1, 'iters': 100} -> Score: 16493.03
iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iterati

Params: {'rng_seed': 42, 'population_size': 10, 'p_increment': 0.1, 'g_increment': 0.1, 'iters': 100} -> Score: 28283.67

Best PSOSA Hyperparameters: {'rng_seed': 42, 'population_size': 10, 'p_increment': 0.1, 'g_increment': 0.1, 'iters': 100}
Best Score: 28283.67457068035

### PSO-SA Training

In [37]:
from optimisers import PSOSA

class TrackingPSOSA(PSOSA):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.score_history = []

    def step(self):
        super().step()
        _, best_fitness = self.current_best()
        self.score_history.append(best_fitness)


In [42]:
import numpy as np
from trading_bot import TradingBotOptimiser
from optimisers import PSOSA

best_psosa_params = {
    'rng_seed': 42,
    'population_size': 10,
    'p_increment': 0.1,
    'g_increment': 0.1,
    'iters': 100
}

train_dataset_path = './trainingDaily.csv'

psosa_optimiser = TrackingPSOSA(
    rng_seed=best_psosa_params['rng_seed'],
    population_size=best_psosa_params['population_size'],
    p_increment=best_psosa_params['p_increment'],
    g_increment=best_psosa_params['g_increment'],
    iters=best_psosa_params['iters']
)

psosa_optim = TradingBotOptimiser(train_dataset_path, psosa_optimiser)
psosa_best_pos, psosa_best_fitness = psosa_optim.optimise(100)
psosa_score_std = np.std(psosa_optimiser.score_history)

print("Best PSOSA Parameters:", best_psosa_params)
print("Train Set Score:", psosa_best_fitness)
print(f"  PSOSA: score {psosa_best_fitness:.8g} at pos ({psosa_best_pos})")
print("Standard Deviation of Scores:", psosa_score_std)


iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Best PSOSA Parameters: {'rng_seed': 42, 'population_size': 10, 'p_increment': 0.1, 'g_increment': 0.1, 'iters': 100}
Train Set Score: 28283.67457068035
  PSOSA: score 28283.675 at pos ([ 7.00000000e+01  1.00000000e+02  4.90000000e+01  6.25333466e-02
 -9.03678234e+01 -9.04913661e+01 -1.00000000e+02 -5.69421064e+00
 -8.26760003e+01  2.99818104e+02])
Standard Deviation of Scores: 3934.1529486536215


Best PSOSA Parameters: {'rng_seed': 42, 'population_size': 10, 'p_increment': 0.1, 'g_increment': 0.1, 'iters': 100}
Train Set Score: 28283.67457068035
  PSOSA: score 28283.675 at pos ([ 7.00000000e+01  1.00000000e+02  4.90000000e+01  6.25333466e-02
 -9.03678234e+01 -9.04913661e+01 -1.00000000e+02 -5.69421064e+00
 -8.26760003e+01  2.99818104e+02])
Standard Deviation of Scores: 3934.1529486536215

# PSO- SA  - test 

In [43]:
from trading_bot import TradingBotInstance

test_dataset_path = './testDaily.csv'

# Use the best parameters found from training
psosa_bot = TradingBotInstance(test_dataset_path, *psosa_best_pos)

# Run simulation on test set
psosa_score = psosa_bot.simulate_run()

print("\nTest Set Evaluation Using Best PSOSA Parameters:")
print("Parameters:", psosa_best_pos)
print("Score:", psosa_score)


Test Set Evaluation Using Best PSOSA Parameters:
Parameters: [ 7.00000000e+01  1.00000000e+02  4.90000000e+01  6.25333466e-02
 -9.03678234e+01 -9.04913661e+01 -1.00000000e+02 -5.69421064e+00
 -8.26760003e+01  2.99818104e+02]
Score: 2211.5269950715974


Test Set Evaluation Using Best PSOSA Parameters:
Parameters: [ 7.00000000e+01  1.00000000e+02  4.90000000e+01  6.25333466e-02
 -9.03678234e+01 -9.04913661e+01 -1.00000000e+02 -5.69421064e+00
 -8.26760003e+01  2.99818104e+02]
Score: 2211.5269950715974

# Artificial Bee Colony - Finding hyper Paramters

In [7]:
import itertools
import numpy as np
from optimisers import PSO, PSOSA, ArtificialBeeColony

class ABCGridSearch:
    def __init__(self, dataset_path, param_grid, iterations):
        self.dataset_path = dataset_path
        self.param_grid = param_grid
        self.iterations = iterations
        self.best_score = -np.inf
        self.best_params = None

    def run(self):
        keys = list(self.param_grid.keys())
        param_combinations = list(itertools.product(*self.param_grid.values()))

        for values in param_combinations:
            params = dict(zip(keys, values))

            optimiser = ArtificialBeeColony(
                rng_seed=params['rng_seed'],
                population_size=params['population_size'],
                pos_age_limit=params['pos_age_limit']
            )

            bot_optim = TradingBotOptimiser(self.dataset_path, optimiser)

            try:
                _, score = bot_optim.optimise(self.iterations)
            except Exception as e:
                print(f"Failed with {params}: {e}")
                continue

            if score > self.best_score:
                self.best_score = score
                self.best_params = params

            print(f"Params: {params} -> Score: {score:.2f}")

        return self.best_params, self.best_score

        
if __name__ == "__main__":
    param_grid = {
        'rng_seed': [42],
        'population_size': [5, 10],
        'pos_age_limit': [3, 5, 7]
    }

    dataset_path = './trainingDaily.csv'
    search = ABCGridSearch(dataset_path, param_grid, 100)
    best_params, best_score = search.run()

    print("\nBest ABC Hyperparameters:", best_params)
    print("Best Score:", best_score)


iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Params: {'rng_seed': 42, 'population_size': 5, 'pos_age_limit': 3} -> Score: 18705.16
iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Params: {'rng_seed': 42, 'population_size': 5, 'pos_age_limit': 5} -> Score: 18499.82
iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteratio

In [ ]:
#Best ABC Hyperparameters: {'rng_seed': 42, 'population_size': 10, 'pos_age_limit': 3}
#Best Score: 25316.08290354025

# Artificial Bee Colony - Train

In [28]:
class TrackingABC(ArtificialBeeColony):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.score_history = []

    def step(self):
        super().step()
        # Get current best from base Optimiser interface
        _, best_fitness = self.current_best()
        self.score_history.append(best_fitness)


In [29]:
from trading_bot import TradingBotOptimiser

best_abc_params = {
    'rng_seed': 42,
    'population_size': 10,
    'pos_age_limit': 3
}

train_dataset_path = './trainingDaily.csv'

# Use tracking subclass
abc_optimiser = TrackingABC(
    rng_seed=best_abc_params['rng_seed'],
    population_size=best_abc_params['population_size'],
    pos_age_limit=best_abc_params['pos_age_limit']
)

abc_optim = TradingBotOptimiser(train_dataset_path, abc_optimiser)

abc_best_pos, abc_best_fitness = abc_optim.optimise(100)

# Compute standard deviation from tracked history
abc_score_std = np.std(abc_optimiser.score_history)

print("Best ABC Parameters:", best_abc_params)
print("Train Set Score:", abc_best_fitness)
print(f"  ABC: score {abc_best_fitness:.8g} at pos ({abc_best_pos})")
print("Standard Deviation of Fitness During Training:", abc_score_std)


iteration: 0
iteration: 5
iteration: 10
iteration: 15
iteration: 20
iteration: 25
iteration: 30
iteration: 35
iteration: 40
iteration: 45
iteration: 50
iteration: 55
iteration: 60
iteration: 65
iteration: 70
iteration: 75
iteration: 80
iteration: 85
iteration: 90
iteration: 95
Best ABC Parameters: {'rng_seed': 42, 'population_size': 10, 'pos_age_limit': 3}
Train Set Score: 25316.08290354025
  PSO: score 25316.083 at pos ([ 70.          63.          56.           0.91669508   7.42379891
 -51.96699202 -59.86903673 -56.09916981  -4.44954893 437.25279956])
Standard Deviation of Fitness During Training: 4262.9775125605565


Best ABC Parameters: {'rng_seed': 42, 'population_size': 10, 'pos_age_limit': 3}
Train Set Score: 25316.08290354025
  PSO: score 25316.083 at pos ([ 70.          63.          56.           0.91669508   7.42379891
 -51.96699202 -59.86903673 -56.09916981  -4.44954893 437.25279956])
Standard Deviation of Fitness During Training: 4262.9775125605565

# Artificial Bee Colony - Test

In [30]:
from trading_bot import TradingBotInstance

test_dataset_path = './testDaily.csv'

# Use the best parameters found from training
abc_bot = TradingBotInstance(test_dataset_path, *abc_best_pos)

# Run simulation on test set
abc_score = abc_bot.simulate_run()

print("\nTest Set Evaluation Using Best PSO Parameters:")
print("Parameters:", abc_best_pos)
print("Score:", abc_score)


Test Set Evaluation Using Best PSO Parameters:
Parameters: [ 70.          63.          56.           0.91669508   7.42379891
 -51.96699202 -59.86903673 -56.09916981  -4.44954893 437.25279956]
Score: 2070.110748663891


Test Set Evaluation Using Best PSO Parameters:
Parameters: [ 70.          63.          56.           0.91669508   7.42379891
 -51.96699202 -59.86903673 -56.09916981  -4.44954893 437.25279956]
Score: 2070.110748663891